# Level-aware Static Scheduler for Data Flow Architecture

## Overview

This notebook explains a Python implementation of a level-aware static scheduler designed for data flow architectures. The scheduler identifies parallel execution opportunities by analyzing dependencies between operations and grouping them into execution levels.

## Key Concepts

### Data Flow Graph

The system is represented as a Directed Acyclic Graph (DAG) where:
- Nodes represent operations or computations
- Edges represent dependencies between operations
- Input nodes have no dependencies
- The absence of cycles ensures a valid execution order

### Execution Levels

Operations are organized into levels based on their dependencies:
- **Level 0**: Operations with no dependencies (inputs)
- **Level N**: Operations whose dependencies are all in levels 0 to N-1
- Operations within the same level can be executed in parallel

# Key Features of the Level-aware Static Scheduler

## Core Functionality
- **Dependency Analysis**: Represents computation as a directed acyclic graph (DAG) where nodes are operations and edges represent dependencies
- **Level-based Execution**: Groups operations into execution levels based on their dependency relationships
- **Parallel Opportunity Detection**: Automatically identifies which operations can be executed concurrently at each level
- **Static Scheduling**: Determines the entire execution schedule before runtime, enabling optimization


## Dependency Graph Definition

The scheduler is built around a directed acyclic graph (DAG) of operations and their dependencies:


In [ ]:
# Level-aware static scheduler for data flow architecture
# Shows parallel execution opportunities at each level

# Define dependencies as a global variable so it can be accessed by all functions
# This dictionary represents a directed acyclic graph (DAG)
# Each key is an operation, and its value is a list of operations it depends on
# For example, 'd' depends on operations 'a' and 'b' to complete before it can execute
dependencies = {
    'input1': [],          # Input nodes have no dependencies
    'input2': [],          # Input nodes have no dependencies
    'a': ['input1'],       # 'a' requires 'input1' to be computed first
    'b': ['input2'],       # 'b' requires 'input2' to be computed first
    'c': ['input1'],       # 'c' requires 'input1' to be computed first
    'd': ['a', 'b'],       # 'd' requires both 'a' and 'b' to be computed first
    'e': ['c'],            # 'e' requires 'c' to be computed first
    'f': ['d'],            # 'f' requires 'd' to be computed first
    'g': ['e', 'f'],       # 'g' requires both 'e' and 'f' to be computed first
    'h': ['d', 'e'],       # 'h' requires both 'd' and 'e' to be computed first
    'i': ['g'],            # 'i' requires 'g' to be computed first
    'j': ['h', 'g'],       # 'j' requires both 'h' and 'g' to be computed first
    'k': ['i', 'j']        # 'k' requires both 'i' and 'j' to be computed first
}

## Main Scheduler Implementation

The core scheduling algorithm identifies and groups operations into execution levels:

In [ ]:
def simulate_level_aware_scheduler():
    """
    Main function that simulates a level-aware static scheduler for a data flow architecture.
    This scheduler identifies operations that can be executed in parallel at each level.

    Returns:
        tuple: Contains (results, execution_levels)
            - results: Dictionary mapping operations to their computed values
            - execution_levels: List of tuples (level_num, operations_at_level, level_results)
    """
    # Define operations and their implementations
    # Each operation is a lambda function that takes a results dictionary
    # and computes its value based on previously computed results
    operations = {
        'input1': lambda r: 5,                      # Initial value, no dependencies
        'input2': lambda r: 3,                      # Initial value, no dependencies
        'a': lambda r: r['input1'] * 2,             # Double the value of input1
        'b': lambda r: r['input2'] + 7,             # Add 7 to input2
        'c': lambda r: r['input1'] ** 2,            # Square input1
        'd': lambda r: r['a'] + r['b'],             # Sum of a and b
        'e': lambda r: r['c'] - 3,                  # Subtract 3 from c
        'f': lambda r: r['d'] / 2,                  # Half of d
        'g': lambda r: r['e'] * r['f'],             # Product of e and f
        'h': lambda r: r['d'] + r['e'],             # Sum of d and e
        'i': lambda r: r['g'] ** 2,                 # Square of g
        'j': lambda r: r['h'] - r['g'],             # Difference of h and g
        'k': lambda r: r['i'] + r['j']              # Sum of i and j
    }

    # Build the reverse dependency graph (which operations depend on each variable)
    # This helps track which operations are affected when a value is computed
    dependent_ops = {}
    for op, deps in dependencies.items():
        for dep in deps:
            if dep not in dependent_ops:
                dependent_ops[dep] = []
            dependent_ops[dep].append(op)

    # Identify execution levels
    # Level 0: operations with no dependencies (inputs)
    # Level N: operations whose dependencies are all in levels 0 to N-1
    results = {}                   # Will store the computed values for each operation
    execution_levels = []          # Will store the execution trace for each level
    remaining_ops = set(operations.keys())  # Track operations that haven't been executed yet

    level = 0
    while remaining_ops:
        current_level = []

        # Find operations that can be executed at this level
        # An operation can be executed if all its dependencies have already been processed
        # (i.e., they're not in the remaining_ops set)
        for op in list(remaining_ops):
            if all(dep not in remaining_ops for dep in dependencies[op]):
                current_level.append(op)

        # If we can't find any operations for this level, there might be a cycle
        # In a valid DAG, we should always find at least one operation if remaining_ops is not empty
        if not current_level:
            print("Error: Possible cycle in dependency graph")
            break

        # Execute operations at this level
        # All operations at the same level can be executed in parallel
        level_results = {}
        for op in current_level:
            try:
                level_results[op] = operations[op](results)  # Execute the operation
                results[op] = level_results[op]              # Store the result
                remaining_ops.remove(op)                     # Mark operation as completed
            except KeyError as e:
                print(f"Error executing {op}: Missing dependency {e}")

        # Store the current level's information for the execution trace
        execution_levels.append((level, current_level, level_results))
        level += 1

    return results, execution_levels

## Execution and Visualization

The scheduler runs and displays the execution trace, highlighting parallel opportunities:

In [ ]:
# Execute the simulation
results, levels = simulate_level_aware_scheduler()

# Print the execution trace with parallel opportunities
print("Static Schedule Execution Trace (Showing Parallel Opportunities):")
print("=" * 70)

# Iterate through each level and display operations that can be executed in parallel
for level_num, ops_at_level, level_results in levels:
    print(f"\nLEVEL {level_num}:")
    print("-" * 70)

    # When multiple operations are at the same level, they can be executed in parallel
    if len(ops_at_level) > 1:
        print(f"The following operations can be executed in parallel at this level:")
        for op in ops_at_level:
            print(f"  - {op} = {level_results[op]}")
    else:
        op = ops_at_level[0]
        print(f"Operation: {op} = {level_results[op]}")



print("\nFinal results:")
for var, value in sorted(results.items()):
    print(f"{var} = {value}")

Static Schedule Execution Trace (Showing Parallel Opportunities):

LEVEL 0:
----------------------------------------------------------------------
The following operations can be executed in parallel at this level:
  - input2 = 3
  - input1 = 5

LEVEL 1:
----------------------------------------------------------------------
The following operations can be executed in parallel at this level:
  - a = 10
  - c = 25
  - b = 10

LEVEL 2:
----------------------------------------------------------------------
The following operations can be executed in parallel at this level:
  - d = 20
  - e = 22

LEVEL 3:
----------------------------------------------------------------------
The following operations can be executed in parallel at this level:
  - h = 42
  - f = 10.0

LEVEL 4:
----------------------------------------------------------------------
Operation: g = 220.0

LEVEL 5:
----------------------------------------------------------------------
The following operations can be executed in pa

## Waiting Behavior Visualization

This function provides a detailed visual representation of operation states at each level:

In [ ]:
# Create a visual demonstration of waiting behavior
def demonstrate_waiting_with_levels():
    """
    Visualizes the execution of operations across different levels,
    showing which operations are computed, which are ready to be computed,
    and which are waiting for dependencies.

    This function helps understand the waiting behavior in a static schedule.
    """
    print("\n" + "=" * 70)
    print("VISUAL DEMONSTRATION OF EXECUTION AND WAITING")
    print("=" * 70)

    # List of all operations in the data flow graph
    all_ops = ['input1', 'input2', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k']
    computed = set()  # Set to track which operations have been computed

    # Iterate through each execution level
    for level_num, ops_at_level, level_results in levels:
        print(f"\nLEVEL {level_num} EXECUTION:")
        print("-" * 70)

        # Add the operations from the current level to the computed set
        computed.update(ops_at_level)

        # For each operation, show its current status:
        # - COMPUTED: Operation has been executed and has a result
        # - READY: All dependencies are available, but not yet executed
        # - WAITING: Some dependencies are not yet available
        for op in all_ops:
            deps = dependencies.get(op, [])
            deps_status = all(dep in computed for dep in deps)

            if op in computed:
                status = f"✓ COMPUTED: {op} = {results[op]}"
            elif deps_status:
                status = f"⏳ READY: {op} (all dependencies available)"
            else:
                missing = [dep for dep in deps if dep not in computed]
                status = f"⏰ WAITING: {op} (missing dependencies: {', '.join(missing)})"

            print(status)

# Run the demonstration to visualize the execution process
demonstrate_waiting_with_levels()


VISUAL DEMONSTRATION OF EXECUTION AND WAITING

LEVEL 0 EXECUTION:
----------------------------------------------------------------------
✓ COMPUTED: input1 = 5
✓ COMPUTED: input2 = 3
⏳ READY: a (all dependencies available)
⏳ READY: b (all dependencies available)
⏳ READY: c (all dependencies available)
⏰ WAITING: d (missing dependencies: a, b)
⏰ WAITING: e (missing dependencies: c)
⏰ WAITING: f (missing dependencies: d)
⏰ WAITING: g (missing dependencies: e, f)
⏰ WAITING: h (missing dependencies: d, e)
⏰ WAITING: i (missing dependencies: g)
⏰ WAITING: j (missing dependencies: h, g)
⏰ WAITING: k (missing dependencies: i, j)

LEVEL 1 EXECUTION:
----------------------------------------------------------------------
✓ COMPUTED: input1 = 5
✓ COMPUTED: input2 = 3
✓ COMPUTED: a = 10
✓ COMPUTED: b = 10
✓ COMPUTED: c = 25
⏳ READY: d (all dependencies available)
⏳ READY: e (all dependencies available)
⏰ WAITING: f (missing dependencies: d)
⏰ WAITING: g (missing dependencies: e, f)
⏰ WAITING: h (

## Benefits of Level-aware Scheduling

1. **Parallelism**: Identifies operations that can safely run concurrently
2. **Resource Optimization**: Groups operations to minimize waiting time
3. **Visualization**: Provides clear insights into execution dependencies
4. **Static Analysis**: Enables optimization before runtime

## Applications

This type of scheduler is useful in:
- Compiler optimization
- Parallel computing systems
- Data processing pipelines
- Task scheduling in distributed systems
- Hardware design for custom processors

## Limitations

- Assumes a static dependency graph (no dynamic dependencies)
- Does not account for operation execution time differences
- Does not handle conditional execution paths